# Science Case #1:

Find all <source-type> within a given area of the sky with emission from
<molecule(s)> detected.


"The science motivation might be to find all young stellar objects (YSOs) in Taurus with ALMA
detections of CO J=2-1. The return from this query would include images of the moment maps of
CO emission, peak intensity, resolution, noise, correlator setting, and other relevant information
from any Taurus YSOs (as identified by science keywords, abstract text, SIMBAD coordinate
matches) in the publicly accessible ALMA Science Archive. The user could identify which data
are appropriate for their study an pull the u,v data or image cubes of interest from the archive.

Optionally, the user could then create a query to retrieve the full set of science products for all
sources, or the sources of interest. This would allow exploration of continuum and other lines
detected in the same observations. The quantitative data are returned as tables in the user’s
Python environment (for example Jupyter notebook) which can then be manipulated (see Section
4), for example to produce a plot of continuum flux versus CO integrated intensity or CO
intensity versus [13CO/12CO] intensity ratio.



In [13]:
from astroquery.admit import ADMIT
from astroquery.alma import Alma, tapsql
from astropy.coordinates import SkyCoord
import pandas as pd
# display the whole table inthe notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',25)
cols_to_exclude = ['id','project_abstract','obs_title','fcoverage']


In [3]:
a = ADMIT()
a.check()
a.key_description.show_in_notebook()


Found  /home/mpound/src/astroumd/study7/query/admit.db
Checking db.... 0
header    : 88 entries
alma      : 88 entries
win       : 88 entries
lines     : 23 entries
sources   : 191 entries


idx,Keyword,Description
0,alma_id,ALMA ID
1,win_id,Spectral window ID
2,spw,Spectral window name
3,nlines,Number of lines
4,nsources,Number of sources
5,nchan,Number of channels
6,win_peak,Window peak flux
7,win_rms,Window RMS noise
8,win_snr,Window Signal to noise ratio (peak/rms)
9,bmaj,Beam major axis


In [4]:
a.keys

['alma_id',
 'win_id',
 'spw',
 'nlines',
 'nsources',
 'nchan',
 'win_peak',
 'win_rms',
 'win_snr',
 'bmaj',
 'bmin',
 'bpa',
 'freqc',
 'freqw',
 'vlsr',
 'fcoverage',
 'spw',
 'restfreq',
 'formula',
 'transition',
 'velocity',
 'vmin',
 'vmax',
 'mom0flux',
 'mom1peak',
 'mom2peak',
 'spw',
 'lines_id',
 'ra',
 'dec',
 'flux',
 'source_snr',
 'source_peak',
 'smaj',
 'smin',
 'spa',
 'header_key',
 'header_val',
 'obs_id',
 'source_name_resolver',
 'source_name_alma',
 'ra_dec',
 'galactic',
 'spatial_resolution',
 'spatial_scale_max',
 'fov',
 'frequency',
 'bandwidth',
 'spectral_resolution',
 'band_list',
 'start_date',
 'integration_time',
 'polarisation_type',
 'line_sensitivity',
 'continuum_sensitivity',
 'water_vapour',
 'project_code',
 'project_title',
 'pi_name',
 'proposal_authors',
 'project_abstract',
 'publication_count',
 'science_keyword',
 'scientific_category',
 'bibcode',
 'pub_title',
 'first_author',
 'authors',
 'pub_abstract',
 'publication_year']

In [5]:
result = a.query(formula="CO", transition="2-1", project_abstract="*Black Hole*")
len(result)

kwargs  {'formula': 'CO', 'transition': '2-1', 'project_abstract': '*Black Hole*'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   inner join lines on (lines.w_id = win.id )  WHERE lines.formula='CO' AND lines.transition='2-1' AND alma.project_abstract LIKE '%Black Hole%' AND sources.l_id > 0 


30

In [6]:
result=a.query(source_name_alma="NGC3049",mom0flux=">0")
len(result) 

kwargs  {'source_name_alma': 'NGC3049', 'mom0flux': '>0'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   inner join lines on (lines.w_id = win.id )  WHERE alma.target_name='NGC3049' AND lines.mom0flux>=0.0 AND sources.l_id > 0 


2

In [7]:
result

,id,obs_id,target_name,s_ra,s_dec,frequency,t_min,project_abstract,obs_title,science_keyword,scientific_category,proposal_authors,id,a_id,spw,freqc,freqw,vlsr,nlines,nsources,nchan,peak_w,rms_w,bmaj,bmin,bpa,fcoverage,id,w_id,l_id,ra,dec,peak_s,flux,smaj,smin,spa,snr_s,id,w_id,formula,transition,restfreq,vmin,vmax,mom0flux,mom1peak,mom2peak
0,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,58067.386351,We propose to use ALM...,Weighing Black Hole M...,"Surveys of galaxies, ...",Galaxy evolution,"Greene, Jenny; Seth, ...",3,3,???,243.83482,1.7031,1447.989431,1,2,109,0.001538,0.00018,0.00001,0.000007,38.894366,0.045872,3,3,2,148.709558,9.268486,0.061878,0.000001,0.027238,0.019856,89.2057,-1.0,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0
1,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,58067.386351,We propose to use ALM...,Weighing Black Hole M...,"Surveys of galaxies, ...",Galaxy evolution,"Greene, Jenny; Seth, ...",3,3,???,243.83482,1.7031,1447.989431,1,2,109,0.001538,0.00018,0.00001,0.000007,38.894366,0.045872,4,3,2,148.708592,9.271231,0.085117,0.000005,0.074912,0.026612,134.7160,-1.0,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0


In [10]:
result=a.query(source_name_alma='NGC3049',formula="*CO*")
len(result)

kwargs  {'source_name_alma': 'NGC3049', 'formula': '*CO*'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   inner join lines on (lines.w_id = win.id )  WHERE alma.target_name='NGC3049' AND lines.formula LIKE '%CO%' AND sources.l_id > 0 


2

In [16]:
result

,id,obs_id,target_name,s_ra,s_dec,frequency,t_min,project_abstract,obs_title,science_keyword,scientific_category,proposal_authors,id,a_id,spw,freqc,freqw,vlsr,nlines,nsources,nchan,peak_w,rms_w,bmaj,bmin,bpa,fcoverage,id,w_id,l_id,ra,dec,peak_s,flux,smaj,smin,spa,snr_s,id,w_id,formula,transition,restfreq,vmin,vmax,mom0flux,mom1peak,mom2peak
0,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,58067.386351,We propose to use ALM...,Weighing Black Hole M...,"Surveys of galaxies, ...",Galaxy evolution,"Greene, Jenny; Seth, ...",3,3,???,243.83482,1.7031,1447.989431,1,2,109,0.001538,0.00018,0.00001,0.000007,38.894366,0.045872,3,3,2,148.709558,9.268486,0.061878,0.000001,0.027238,0.019856,89.2057,-1.0,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0
1,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,58067.386351,We propose to use ALM...,Weighing Black Hole M...,"Surveys of galaxies, ...",Galaxy evolution,"Greene, Jenny; Seth, ...",3,3,???,243.83482,1.7031,1447.989431,1,2,109,0.001538,0.00018,0.00001,0.000007,38.894366,0.045872,4,3,2,148.708592,9.271231,0.085117,0.000005,0.074912,0.026612,134.7160,-1.0,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0


### Example of how to exclude columns listed in the first cell

In [15]:
#columns to include 
ci = [x for x in result.columns if x not in cols_to_exclude]
result[ci]

,obs_id,target_name,s_ra,s_dec,frequency,t_min,science_keyword,scientific_category,proposal_authors,a_id,spw,freqc,freqw,vlsr,nlines,nsources,nchan,peak_w,rms_w,bmaj,bmin,bpa,w_id,w_id,l_id,ra,dec,peak_s,flux,smaj,smin,spa,snr_s,w_id,w_id,formula,transition,restfreq,vmin,vmax,mom0flux,mom1peak,mom2peak
0,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,58067.386351,"Surveys of galaxies, ...",Galaxy evolution,"Greene, Jenny; Seth, ...",3,???,243.83482,1.7031,1447.989431,1,2,109,0.001538,0.00018,0.00001,0.000007,38.894366,3,3,2,148.709558,9.268486,0.061878,0.000001,0.027238,0.019856,89.2057,-1.0,3,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0
1,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,58067.386351,"Surveys of galaxies, ...",Galaxy evolution,"Greene, Jenny; Seth, ...",3,???,243.83482,1.7031,1447.989431,1,2,109,0.001538,0.00018,0.00001,0.000007,38.894366,3,3,2,148.708592,9.271231,0.085117,0.000005,0.074912,0.026612,134.7160,-1.0,3,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0


In [17]:
result=a.query(source_name_alma='NGC3504',formula="CS")
len(result)

kwargs  {'source_name_alma': 'NGC3504', 'formula': 'CS'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   inner join lines on (lines.w_id = win.id )  WHERE alma.target_name='NGC3504' AND lines.formula='CS' AND sources.l_id > 0 


3

In [18]:
result[ci]

,obs_id,target_name,s_ra,s_dec,frequency,t_min,science_keyword,scientific_category,proposal_authors,a_id,spw,freqc,freqw,vlsr,nlines,nsources,nchan,peak_w,rms_w,bmaj,bmin,bpa,w_id,w_id,l_id,ra,dec,peak_s,flux,smaj,smin,spa,snr_s,w_id,w_id,formula,transition,restfreq,vmin,vmax,mom0flux,mom1peak,mom2peak
0,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,243.689957,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",32,???,243.689976,1.867059,1521.106704,1,1,478,0.012112,0.001397,0.000174,0.000118,4.293194,32,32,20,165.796838,27.972631,0.470603,0.003008,0.632991,0.313058,148.896,-1.0,32,32,CS,5-4,244.93556,-66.2128,-13.3561,140.5190,1483.46,15.8738
1,uid://A001/X87a/X708,NGC3504,165.796629,27.972436,243.690784,57830.136178,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",35,???,243.690785,1.867264,1521.106704,1,1,478,0.023037,0.002596,0.000550,0.000420,9.199411,35,35,21,165.796779,27.972628,1.596050,0.120837,2.175080,1.079050,158.474,-1.0,35,35,CS,5-4,244.93556,-43.1873,57.7322,107.7540,1538.80,31.0504
2,uid://A001/X87a/X70a,NGC3504,165.796629,27.973328,243.689992,57704.450990,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",38,???,243.690031,1.867093,1521.106704,1,1,478,0.043824,0.005212,0.001621,0.001531,51.806499,38,38,23,165.796767,27.972697,3.970890,2.056540,4.346940,3.693410,157.490,-1.0,38,38,CS,5-4,244.93556,-42.2564,82.6801,91.1055,1544.31,24.9688


In [19]:
result1 = a.query(source_name_alma="NGC3504")
#result2 = a.query(source_name_resolver='ngc3504')
sql = "select * from alma where alma.target_name='NGC3504'"
result2 = a.sql(sql)
result2=pd.DataFrame(result2,columns=a._colnames['alma'])
result3 = a.query(source_name_alma="NGC3504",mom0flux=">-1")

kwargs  {'source_name_alma': 'NGC3504'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   WHERE alma.target_name='NGC3504' AND sources.l_id = 0 
kwargs  {'source_name_alma': 'NGC3504', 'mom0flux': '>-1'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   inner join lines on (lines.w_id = win.id )  WHERE alma.target_name='NGC3504' AND lines.mom0flux>=-1.0 AND sources.l_id > 0 


In [ ]:
print(len(result1),len(result2),len(result3))

In [ ]:
result1[ci]

In [20]:
result3[ci]

,obs_id,target_name,s_ra,s_dec,frequency,t_min,science_keyword,scientific_category,proposal_authors,a_id,spw,freqc,freqw,vlsr,nlines,nsources,nchan,peak_w,rms_w,bmaj,bmin,bpa,w_id,w_id,l_id,ra,dec,peak_s,flux,smaj,smin,spa,snr_s,w_id,w_id,formula,transition,restfreq,vmin,vmax,mom0flux,mom1peak,mom2peak
0,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.796846,27.972625,38.060800,0.968566,1.085610,0.726671,160.468000,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
1,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.796750,27.973208,8.522460,1.126240,2.316410,1.768540,19.418700,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
2,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.796413,27.972947,5.330860,0.025489,0.385000,0.385000,180.000000,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
3,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.796562,27.972433,5.362120,0.025639,0.385000,0.385000,90.000000,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
4,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.796471,27.972986,5.237260,0.782512,2.988290,1.549980,18.780700,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
5,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.796221,27.971853,4.362290,0.161440,1.225940,0.935810,167.332000,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
6,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.797192,27.971889,3.892510,0.132583,1.322840,0.798201,13.075000,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
7,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.796017,27.972922,3.836720,0.133965,1.495380,0.723835,177.936000,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
8,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.001223,0.000185,0.000126,4.444983,30,30,19,165.795746,27.973592,3.668560,0.149577,1.732620,0.729502,28.806100,-1.0,30,30,CO,2-1,230.53800,-158.9050,183.1460,24782.4000,1548.35,60.9915
9,uid://A001/X87a/X706,NGC3504,165.796629,27.972436,229.366802,57713.452563,Galactic centres/nuclei,Active galaxies,"Trejo, Alfonso; Jiang...",30,???,229.366820,-1.867056,1521.106704,1,30,478,0.253098,0.00122

In [21]:
result=a.query(scientific_category="galaxy evolution")  # case sensitive! returns zero results
len(result)

kwargs  {'scientific_category': 'galaxy evolution'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   WHERE alma.scientific_category='galaxy evolution' AND sources.l_id = 0 


0

In [22]:
result=a.query(win_snr=">1",source_name_alma="NGC3049") 
len(result)

kwargs  {'win_snr': '>1', 'source_name_alma': 'NGC3049'}
select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id)   WHERE ( win.peak_w / win.rms_w )>=1.0 AND alma.target_name='NGC3049' AND sources.l_id = 0 


6

In [25]:
sql = "select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id) join lines on (lines.w_id = win.id) WHERE alma.target_name='NGC3504' and (sources.l_id > 0)"
result4_linecube = pd.DataFrame(data=a.sql(sql),columns=a._colnames['alma']+a._colnames['win']+a._colnames['sources']+a._colnames['lines'])

sql = "select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id) join lines on (lines.w_id = win.id) WHERE alma.target_name='NGC3504' and (sources.l_id = 0)"
result4_cubesum = pd.DataFrame(data=a.sql(sql),columns=a._colnames['alma']+a._colnames['win']+a._colnames['sources']+a._colnames['lines'])
len(result4_linecube),len(result4_cubesum)

(34, 34)

In [27]:
list(zip(result4_linecube.flux,result4_cubesum.flux))


[(0.968566, 97.1),
 (1.12624, 118.0),
 (0.0254893, 2.7),
 (0.0256387, 2.67),
 (0.782512, 83.9),
 (0.16144, 16.5),
 (0.132583, 13.8),
 (0.133965, 14.0),
 (0.149577, 16.1),
 (0.0159732, 1.76),
 (0.103948, 10.7),
 (0.452096, 48.6),
 (0.547587, 61.3),
 (0.0137423, 1.43),
 (0.0986551, 10.5),
 (0.0110917, 1.3),
 (0.115968, 11.7),
 (0.0688029, 7.35),
 (0.206304, 22.8),
 (0.00807528, 0.958),
 (0.0623572, 6.45),
 (0.0753269, 8.44),
 (0.11688, 13.1),
 (0.12272, 13.3),
 (0.25279, 27.8),
 (0.117098, 12.0),
 (0.107637, 11.5),
 (0.0681595, 7.64),
 (0.0082443, 0.839),
 (0.211353, 23.2),
 (0.00300826, 0.547),
 (0.120837, 0.934),
 (1102.06, 927.0),
 (2.05654, 1.65)]

In [28]:
sql = "select * from alma inner join win on (win.a_id = alma.id) inner join sources on (sources.w_id = win.id) WHERE alma.target_name='NGC3504' and sources.l_id = 0 "
result5 = pd.DataFrame(data=a.sql(sql),columns=a._colnames['alma']+a._colnames['win']+a._colnames['sources'])
len(result5)

50

In [ ]:
result5

-------------------
tinker code below

In [ ]:
_gen_numeric_sql = _gen_str_sql = _gen_pos_sql = _gen_spec_res_sql = _gen_band_list_sql = _gen_datetime_sql = _gen_pol_sql = None

In [ ]:
ADMIT_FORM_KEYS = {
    'Window': {  
        'ALMA ID': ['alma_id','win.alma_id',_gen_numeric_sql],
        'Spectral Window': [ 'spw','win.id', _gen_numeric_sql],
        'Number of Lines': [ 'nlines','win.nlines', _gen_numeric_sql],
        'Number of Sources': [ 'nsources','win.nsources', _gen_numeric_sql],
        'Number of Channels': [ 'nchan','win.nchan', _gen_numeric_sql],
        'RMS noise': [ 'rms','win.rms', _gen_numeric_sql],
     },
    'Lines': {
        'Spectral Window': [ 'spw','lines.w_id', _gen_numeric_sql],
        'Formula': [ 'formula','lines.formula', _gen_str_sql],
        'Transition': [ 'transition','lines.transition', _gen_str_sql],
        'Velocity': [ 'velocity','lines.velocity', _gen_numeric_sql],
        # we are not using this
        #'Channels': [ 'chan','lines.chan', _gen_numeric_sql],
     },
    'Sources': {
        'Spectral Window': [ 'spw','sources.w_id', _gen_numeric_sql],
        'Line ID': [ 'lines_id','sources.lines_id', _gen_numeric_sql],
        'RA (Degrees)': ['ra', 'sources.ra',  _gen_numeric_sql],
        'Dec (Degrees)': ['dec', 'sources.dec',  _gen_numeric_sql],
        'Flux': ['flux', 'sources.flux',  _gen_numeric_sql],
        # source.snr is not actually a table column but we can use it as 
        # a trigger to munge some sql post-facto.
        'Signal to Noise Ratio': ['snr', 'sources.snr',  _gen_numeric_sql],
     },
    'Header': { # no science use case
        'Key': ['header_key','header.key',_gen_str_sql],
        'Value': ['header_val','header.val',_gen_str_sql],
     },
    'Cont': {
        'ALMA ID': ['alma_id','cont.alma_id',_gen_numeric_sql],
        'Bands': ['band','cont.cont',_gen_str_sql],
        'Number of Continuum Sources': ['nc_sources','cont.nsources',_gen_numeric_sql],
     },
    'Alma': {
        'Observation': ['obs_id','alma.obs_id',_gen_str_sql],
        # From here below are just a copy of ALMA_FORM_KEYS without the external wrapper dict.
        # Position
        'Source name (astropy Resolver)': ['source_name_resolver',
                                           'SkyCoord.from_name', _gen_pos_sql],
        'Source name (ALMA)': ['source_name_alma', 'alma.target_name', _gen_str_sql],
        'RA Dec (Sexagesimal)': ['ra_dec', 'alma.s_ra, alma.s_dec', _gen_pos_sql],
        'Galactic (Degrees)': ['galactic', 'alma.gal_longitude, alma.gal_latitude',
                               _gen_pos_sql],
        'Angular resolution (arcsec)': ['spatial_resolution',
                                        'alma.spatial_resolution', _gen_numeric_sql],
        'Largest angular scale (arcsec)': ['spatial_scale_max',
                                           'alma.spatial_scale_max', _gen_numeric_sql],
        'Field of view (arcsec)': ['fov', 'alma.s_fov', _gen_numeric_sql],
        # Energy
        'Frequency (GHz)': ['frequency', 'alma.frequency', _gen_numeric_sql],
        'Bandwidth (Hz)': ['bandwidth', 'alma.bandwidth', _gen_numeric_sql],
        'Spectral resolution (KHz)': ['spectral_resolution',
                                      'alma.em_resolution', _gen_spec_res_sql],
        'Bandesult = a.query(formula="CO", transition="2-1")': ['band_list', 'alma.band_list', _gen_band_list_sql],
        # Time
        'Observation date': ['start_date', 'alma.t_min', _gen_datetime_sql],
        'Integration time (s)': ['integration_time', 'alma.t_exptime',
                                 _gen_numeric_sql],
        # Polarization
        'Polarisation type (Single, Dual, Full)': ['polarisation_type',
                                                   'alma.pol_states', _gen_pol_sql],
        # Observation
        'Line sensitivity (10 km/s) (mJy/beam)': ['line_sensitivity',
                                                  'alma.sensitivity_10kms',
                                                  _gen_numeric_sql],
        'Continuum sensitivity (mJy/beam)': ['continuum_sensitivity',
                                             'alma.cont_sensitivity_bandwidth',
                                             _gen_numeric_sql],
        'Water vapour (mm)': ['water_vapour', 'alma.pvw', _gen_numeric_sql],
        # Project
        'Project code': ['project_code', 'alma.proposal_id', _gen_str_sql],
        'Project title': ['project_title', 'alma.obs_title', _gen_str_sql],
        'PI name': ['pi_name', 'alma.obs_creator_name', _gen_str_sql],
        'Proposal authors': ['proposal_authors', 'alma.proposal_authors', _gen_str_sql],
        'Project abstract': ['project_abstract', 'alma.proposal_abstract', _gen_str_sql],
        'Publication count': ['publication_count', 'alma.NA', _gen_str_sql],
        'Science keyword': ['science_keyword', 'alma.science_keyword', _gen_str_sql],
       # Publication'
        'Bibcode': ['bibcode', 'alma.bib_reference', _gen_str_sql],
        'Title': ['pub_title', 'alma.pub_title', _gen_str_sql],
        'First author': ['first_author', 'alma.first_author', _gen_str_sql],
        'Authors': ['authors', 'alma.authors', _gen_str_sql],
        # this may need special handling? or person does search on pub_abstract="*YSO*" 
        'Abstract': ['pub_abstract', 'alma.pub_abstract', _gen_str_sql],
        'Year': ['publication_year', 'alma.pub_year', _gen_numeric_sql],
     },
}

In [ ]:
s = ['formula', 'lines.formula', None]
s in ADMIT_FORM_KEYS["Lines"].values()

In [ ]:
ADMIT_FORM_KEYS["Lines"].values()

In [ ]:
if False:
    result1 = Alma.query_tap("select * from ivoa.obscore WHERE target_name = 'NGC3504'")
    result2 = Alma.query_tap("select * from ivoa.obscore WHERE target_name = 'ngc3504'")
    coord = SkyCoord.from_name('ngc3504')
    print(coord)
    sqlcoord = tapsql._gen_pos_sql('SkyCoord.from_name','ngc3504')
    print(sqlcoord)
    sql = "select * from ivoa.obscore WHERE source_name_resolver = '" +sqlcoord+"'"
    print(sql)
    if False:
        result3 = Alma.query_tap(sql) # fails    